In [ ]:
import numpy as np
#import matplotlib.pyplot as plt

In [ ]:
class MLP:

  def __init__( self, sizes):
    self.S = sizes
    self.L = len(self.S)
    self.Y = []
    self.W = [0]
    self.dW = [0]
  
  def bias_add(self,V):
    bias = -np.ones( (len(V),1) )
    return np.concatenate( (V,bias), axis=1)

  def bias_sub(self,V):
    return V[:,:-1]

  def activation(self,X):
    
    for k in range(1,self.L-1):
        self.Y[k][:] = self.bias_add( np.tanh( np.dot(self.Y[k-1],self.W[k]) ) )
        
    self.Y[-1][:] = np.tanh( np.dot(self.Y[-2],self.W[-1]) )
    #return self.Y

  def correction(self,Z,lr):
    self.dW = [0]
    for i in range(1,self.L): 
        self.dW.append( np.zeros( (self.S[i-1]+1,self.S[i]) ) )
         
    E = [0]*self.L # size L 
    D = [0]*self.L # size L
    d = [0]*self.L # size L
    
    E[self.L - 1] =  Z - self.Y[-1] 
    d[self.L - 1] = 1 - self.Y[-1]**2
    D[self.L - 1] = E[-1]*d[-1] 
        
    for k in reversed(np.arange(1,self.L)):
        self.dW[k] = lr * np.dot( self.Y[k-1].T, D[k]) # lr = learning rate (se le pasa a train)
        E[k] = np.dot( D[k], self.W[k].T )
        d[k] = ( 1 - self.Y[k-1]**2 )
        D[k-1] = self.bias_sub( (E[k]*d[k]) )
    #return self.dW
  
  def adaptation(self):
    
    for k in range(1,self.L-1):
      self.W[k] += self.dW[k]
    #return self.W

  def estimation(self,Z):

    est = np.mean( np.square(Z - self.Y[-1]) )
    return est

  def train(self,X, Z, lr, eps, epoch):

    i = 0
    P = len(X)
    for s in self.S: # Armo las capas
        if i == self.L - 1:
            self.Y.append( np.zeros((P,s)) )
            i += 1
        else:
            self.Y.append( np.zeros((P,s+1)) )
            i += 1
        
    self.Y[0][:] = self.bias_add(X)

    for i in range(0,self.L-1): # Armo los W
      w_i = np.random.normal( 0, 0.1, (self.S[i]+1,self.S[i+1]) )
      self.W.append(w_i)

    error = 1
    iter = 0
    errores = []
    while error > eps and iter < epoch:

      self.activation(X)
      self.correction(Z,lr)
      self.adaptation()
      error = self.estimation(Z)

      errores.append(error)
      iter += 1

    return errores

  def predict(self,X):
    
    Y = []
    P = len(X)
    i = 0
    for s in self.S: # Armo las capas
        if i == self.L - 1:
            Y.append( np.zeros((P,s)) )
            i += 1
        else:
            Y.append( np.zeros((P,s+1)) )
            i += 1
        
    Y[0] = self.bias_add(X)
    for k in range(1,self.L-1):
        Y[k] = self.bias_add( np.tanh( np.dot(Y[k-1],self.W[k]) ) )
        
    Y[-1] = np.tanh( np.dot(Y[-2],self.W[-1]) )
    
    return Y[-1]